In [2]:
import numpy  as np
import torch
from ase.units import kB

Read potential energies.

In [3]:
def cv(E,T):
    E = E.flatten()
    # eV**2 / ((eV/K) * K ** 2) -> eV/K
    return (np.mean((E - np.mean(E))**2))/(kB*T**2)

def CEA(obs, pot, T):

    #assuming obs is an array of shape ( N_Prop_dim, N_samples)
    #and pot is an array of shape ( N_samples, N_ens)
    # T is the temperature in K
    # returns N_ensemble weighted average obs

    beta = 1/(kB*T)
    w_obs = []
    
    obs_mean = np.mean(obs, axis=1) # (N, 1)
    pot_mean = np.mean(pot, axis=1) 


    #print(obs_mean.shape)
    #print(pot_mean.shape)

    for i in range(pot.shape[-1]):
        #print(np.mean(obs * (pot[:,i] - pot_mean),axis=1).shape)
        #print((pot[:,i] - pot_mean).shape)
        #print((np.mean(pot[:,i] - pot_mean)).shape)

        w_obs_i = obs_mean - beta * (np.mean(obs * (pot[:,i] - pot_mean),axis=1) - obs_mean * np.mean(pot[:,i] - pot_mean))
        w_obs.append(w_obs_i)

    return w_obs 


def get_cv_from_CEA(E, T):
    #assuming CEA is an array of shape (N_ensemble, N_Prop_dim)
    #and T is the temperature in K
    # returns N_ensemble weighted average cv

    beta = 1/(kB*T)
    cv = []

    weighted_E = CEA(np.mean(E,axis=1).reshape(-1,1).T, E, T)
    weighted_E_squared = CEA(np.mean(E**2,axis=1).reshape(-1,1).T, E, T)
    #⟨𝐸2⟩−⟨𝐸⟩2
    
    for e_i, e_i_squared in zip(weighted_E, weighted_E_squared):
        cv.append((e_i_squared - e_i**2)/kB*T**2)
        
    return cv


In [215]:
Temp = 300
N_H2O = 128



U_mean = np.mean(E_POTS[Temp][N_H2O][500:],axis=1).reshape(-1,1)

out_energy = np.stack(CEA(U_mean.T, E_POTS[Temp][N_H2O][500:], Temp)) # Nsamples, Ndim
out_energy_squared = np.stack(CEA(U_mean.T**2, E_POTS[Temp][N_H2O][500:], Temp)) #Nsamples, Ndim

heat_coeffs = []


for E_i, E_i_sq in zip(out_energy, out_energy_squared):
    cv_i = ((E_i_sq - E_i**2)/(kB*Temp**2))/(N_H2O)/kB
    heat_coeffs.append(float(cv_i))

print("heat_coeff: {:.2f} pm {:.2f}".format(np.mean(heat_coeffs),np.std(heat_coeffs)))

heat_coeff: 2.72 pm 9.10


/var/folders/7y/yhl65v0j5m3g8zj_xwkzq44w0000gn/T/ipykernel_9001/2708824120.py:14: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  heat_coeffs.append(float(cv_i))
